In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support import expected_conditions as EC
import time
import csv
import os

In [ ]:
#utils

def parseRecord(record):
    """
    Returns:
    str, str, str, str: TrackCode, LawFirm, AttnName, AttnDocket
    """

    TrackCode = record["TrackCode"]
    LawFirm = record["Firm"]
    AttnName = record["AttnName"]
    AttnDocket = record["AttnDocket"]

    return TrackCode, LawFirm, AttnName, AttnDocket

def getTableData(driver, isInvention, wait_time=15):
    """
    Parameters:
        driver (WebDriver): The Selenium WebDriver instance.
        invention (bool): If True, fetches invention attorney data; otherwise, fetches intellectual property attorney data.
    """
    # Wait for the table to be present in the DOM
    

    if isInvention:
        WebDriverWait(driver, wait_time).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, 'table[data-testing-id="listtable_inventionattorney"]'))
        )
        table = driver.find_element(By.CSS_SELECTOR, 'table[data-testing-id="listtable_inventionattorney"]')
    else:
        WebDriverWait(driver, wait_time).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, 'table[data-testing-id="listtable_patentattorney"]'))
        )
        table = driver.find_element(By.CSS_SELECTOR, 'table[data-testing-id="listtable_patentattorney"]')
    return table



def addFirm(driver, record, isInvention=True, wait_time=15):
    TrackCode, LawFirm, AttnName, AttnDocket = parseRecord(record)

    print("Adding Firm... ",end="")

    driver.find_element(By.ID, "name_search_id").send_keys(LawFirm + Keys.ENTER)
    WebDriverWait(driver, wait_time).until(
        EC.presence_of_element_located((By.XPATH,'//input[starts-with(@name, "add_link_checklist::")]'))
    ).click()
    WebDriverWait(driver, wait_time).until(
        EC.presence_of_element_located((By.NAME, "add_link"))
    ).click()

    complete_attorney(driver, record, isInvention)

    finish_edit(driver)


def modifyFirm(driver, record, isInvention=True):
    #assumes the lawfirm is correct

    print("Modifying Firm... ",end="")

    complete_attorney(driver, record, isInvention)

    finish_edit(driver)




def finish_edit(driver):
    driver.find_element(By.NAME, "save").click()
    time.sleep(2)
    print("Done")
    return True

In [ ]:
def checkCurrentAttorneys(driver, record, isInvention=True):
    #Is there a need to distinguish invention and intellectual property? Yes for adding firms I think

    TrackCode, LawFirm, AttnName, AttnDocket = parseRecord(record)

    print("Checking Attorneys... ",end="")

    # Case 1: need to add
    if(driver.find_elements(By.CLASS_NAME, "zero-records")): # no firm at all
        return 1
    
    # Locate the table by its data-testing-id
    table = getTableData(driver, isInvention) 


    # Loop through all rows in the table body
    rows = table.find_elements(By.XPATH, './/tbody/tr')

    for row in rows:
        try:
            # Get the attorney firm name from the 2nd column (td[2])
            firm_elem = row.find_element(By.XPATH, './/td[2]/a')
            firm_name = firm_elem.text.strip()

            # Case 2: need to modify (any combination of (primary, name, docket)). Assumes lawfirm is correct
            if firm_name == LawFirm:
                return 2
        except Exception as e:
            print(f"⚠️ checkCurrentAttorneys method. Skipping row due to error: {e}")
    
    return 1 # no correct firm found



In [ ]:
def complete_attorney(driver, record, isInvention=True, wait_time=15): 
    """
    Marks the correct attorny as primary, fills in AttnName and AttnDocket if not already filled.
    Returns True if successful.
    """

    TrackCode, LawFirm, AttnName, AttnDocket = parseRecord(record)

    # Locate the table by its data-testing-id
    table = getTableData(driver, isInvention)

    # Loop through all rows in the table body
    rows = table.find_elements(By.XPATH, './/tbody/tr')
    for row in rows:
        try:
            # Get the attorney firm name from the 2nd column (td[2])
            firm_elem = row.find_element(By.XPATH, './/td[2]/a')
            firm_name = firm_elem.text.strip()

            # If it matches our target firm
            if firm_name == LawFirm:
                if not isInvention:
                    #for intellectual property
                    primary_radio = row.find_element(By.CSS_SELECTOR, 'input[data-testing-id="primary"]')
                    primary_radio.click()
                    
                    field = row.find_element(By.XPATH, '//select[starts-with(@name, "person_id_")]')
                    selectfield = Select(field)
                    if  ("None" == selectfield.first_selected_option.text.strip()):
                        field.send_keys(AttnName)
                  

                    field = row.find_element(By.CSS_SELECTOR, "input.PatentAttorney-attorney_reference_number")
                    if  (not field.get_attribute("value").strip()):
                        field.send_keys(AttnDocket)

                    return True
                if isInvention:
                    # Click the primary radio button in the 1st column (td[1])
                    primary_radio = row.find_element(By.CSS_SELECTOR, 'input[data-testing-id="is_primary"]')
                    primary_radio.click()
                    
                    field = row.find_element(By.XPATH, '//select[starts-with(@name, "person_id_")]')
                    selectfield = Select(field)
                    if  ("None" == selectfield.first_selected_option.text.strip()):
                        field.send_keys(AttnName)

                    field = row.find_element(By.CSS_SELECTOR, 'input.InventionAttorney-attorney_reference_number')
                    if  (not field.get_attribute("value").strip()):
                        field.send_keys(AttnDocket)
                    elif field.get_attribute("value").strip() == "TBD":
                        field.clear()
                        field.send_keys(AttnDocket)

                    return True  
            
        except Exception as e:
            print(f"⚠️ complete_attorney method. Skipping row due to error: {e}")
            exit(1)


In [ ]:
def setUp():
    print("Setting up Selenium...", end="")
    # === Set up Selenium ===
    driver = webdriver.Chrome()  # make sure chromedriver is installed and in PATH
    driver.get("https://case.wellspringsoftware.net/kms/")

    print("Logging into Soifa...", end="")
    # === Log in if needed ===
    time.sleep(2)
    driver.find_element(By.XPATH, '//*[@id="username"]').send_keys("") #username
    driver.find_element(By.XPATH, '//*[@id="password"]').send_keys("") #password
    driver.find_element(By.ID, "loginButton").click()
    
    print("Please approve duo mobile...", end="")
    #Duo Mobile approve
    #waits until the global search field is located before returning true. Verifies that the page is loaded.
    WebDriverWait(driver, 60).until(
        EC.visibility_of_element_located((By.NAME, "search_field"))
        
    )

    print("Setup complete.")
    return driver
    

In [ ]:
def find_invention(driver, record, wait_time=15):
    TrackCode, LawFirm, AttnName, AttnDocket = parseRecord(record)

    print(f"Finding Invention {TrackCode}... ", end="")
    #  Find Invention
    #go to record find form page
    driver.get("https://case.wellspringsoftware.net/kms/find/form/?find_page=default")

    #look up trackcode
    driver.find_element(By.XPATH, '//*[@id="oform:: 5"]').send_keys(TrackCode + Keys.ENTER)

    #enter invention
    WebDriverWait(driver, wait_time).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="search_results"]/tbody/tr/td[1]')) 
    ).find_element(By.TAG_NAME, "a").click() # click first item in search results

    time.sleep(2)

In [ ]:
def edit_invention(driver, record, wait_time=15):

    print(f"Editing invention... ", end="")

    #  Edit Invention Attn
    WebDriverWait(driver, wait_time).until(
        EC.presence_of_element_located((By.XPATH, '//img[@alt="Edit Invention Attorneys"]/ancestor::a'))
    ).click() #click edit attorneys


    match (checkCurrentAttorneys(driver, record, isInvention=True)):
        case 1: 
            addFirm(driver, record, isInvention=True)
        case 2: 
            modifyFirm(driver, record, isInvention=True)
        case _:
            print("Error with checkCurrentAttorneys. Not case 1 or case 2")
            exit(1)

    



In [ ]:
def edit_intellectualProperty(driver, record, wait_time=15):
    print(f"Editing IP... ", end="")

    #  Edit IP Attn
    WebDriverWait(driver, wait_time).until(
        EC.presence_of_element_located((By.XPATH, '//img[@alt="Edit Patent Attorneys"]/ancestor::a'))
    ).click() #click edit attorneys

    match (checkCurrentAttorneys(driver, record, isInvention=False)):
        case 1: 
            addFirm(driver, record, isInvention=False)
        case 2: 
            modifyFirm(driver, record, isInvention=False)
        case _:
            print("Error with checkCurrentAttorneys. Not case 1 or case 2")
            exit(1)
    

In [ ]:
def saveIP_links(driver, record, filename="IP_links.csv", wait_time=15):

    # Locate the table by the data-testing-id attribute
    WebDriverWait(driver, wait_time).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "table[data-testing-id='invention_financialsnapshot']"))
    ) 
    table = driver.find_element(By.CSS_SELECTOR, "table[data-testing-id='invention_financialsnapshot']")

    # Get all rows in tbody
    rows = table.find_elements(By.CLASS_NAME, "agreement_financial_snapshot_row")


    entries = []

    print("Reading IP links... ", end="")
    for row in rows:
        # Find all anchor tags in the current row
        anchors = row.find_elements(By.TAG_NAME, "a")
        # Extract href attribute from each anchor
        href = anchors[0].get_attribute("href") if anchors else None
        
        #add a counter in case of wifi failure
        record['link'] = href
        entries.append(record.copy())
  
    

    print("Writing IP links... ", end="")
    fieldnames = entries[0].keys()

    with open(filename, 'w', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()  # Write the header row
        writer.writerows(entries) # Write all the dictionary rows
    
    print("Complete")
    return True



In [ ]:
def update_law_firm_invention(driver, records, wait_time=15):
    """
    Updates the law firm information for each record in records.
    """

    print("Updating Law Firm Invention... ")
    
    index = 1
    for record in records:
        find_invention(driver, record, wait_time=wait_time)
        edit_invention(driver, record, wait_time=wait_time)
        saveIP_links(driver, record, filename=f"Invention_IPs/{record["TrackCode"]}_{index}.csv", wait_time=wait_time)
        index += 1

    print("Invention Law Firms updated successfully.")


In [ ]:
def update_law_firm_intellectual_property(driver, wait_time=15, folderpath="Invention_IPs"):
    for Invention_file in os.listdir(folderpath): #for each file in folder

        df = []

        with open(os.path.join(folderpath, Invention_file), newline='') as csvfile:
            reader = csv.DictReader(csvfile)
            for row in reader:
                df.append(row)

        print(f"Processing file: {Invention_file}")
        for IntellectualProperty in df: #for each ip in file
            try:
                if "invention" in IntellectualProperty['link']:
                    continue
                # Open link in Selenium
                driver.get(IntellectualProperty['link'])

                WebDriverWait(driver, wait_time).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "table[data-testing-id='invention_financialsnapshot']"))
                ) 
                
                # === Do your Selenium actions here ===
                print(f"Opened: {IntellectualProperty['link']}")
                edit_intellectualProperty(driver, IntellectualProperty)
                    
            except Exception as e:
                print(f"Error opening {IntellectualProperty['link']}: {e}")
    
    print(f"Intellectual properties in {folderpath} directory updated.")

In [ ]:
#Main

# #open and store data to be updated
    # "TrackCode":"2014-2510",
    # "Firm":"Greenberg Traurig, LLP",
    # "AttnName":"Chinh H. Pham",
    # "AttnDocket":"189573-010100"
inventions = []
with open('lawfirm.csv', newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        inventions.append(row)


# === Set up Selenium ===
driver = setUp() 

update_law_firm_invention(driver, inventions, wait_time=15)
update_law_firm_intellectual_property(driver, wait_time=15)


# === Done ===
driver.quit()

In [ ]:
def open_inventions_in_new_tab(driver, TrackCode, wait_time=15):

    print(f"Finding Invention {TrackCode}... ", end="")
    #  Find Invention
    #go to record find form page
    driver.switch_to.new_window('tab')
    driver.get("https://case.wellspringsoftware.net/kms/find/form/?find_page=default")

    #look up trackcode
    driver.find_element(By.XPATH, '//*[@id="oform:: 5"]').send_keys(TrackCode + Keys.ENTER)

    #enter invention
    WebDriverWait(driver, wait_time).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="search_results"]/tbody/tr/td[1]')) 
    ).find_element(By.TAG_NAME, "a").click() # click first item in search results

    time.sleep(2)


In [ ]:
def open_invention_list():
    list = [    
        "2012-2163",
        "2012-2163",
        "2017-3114",
        "2017-3161",
        "2018-3436",
        "2012-2163",
        "2017-3114",
        "2018-3415",
        "2019-3533",
        "2021-3892"
        ]

    driver = setUp()

    for trackcode in list:
        open_inventions_in_new_tab(driver, trackcode, wait_time=15)